In [1]:
import pandas as pd
from tqdm import tqdm

In [8]:
BASE_PATH = 'C:/Users/lubaude/Documents/Datagrosyst/git_pretraitements/'
#ENTREPOT_PATH = '~/Bureau/utils/data/'
ENTREPOT_PATH = 'C:/Users/lubaude/Desktop/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [5]:
donnees = {}

def import_df(df_name, path_data, sep, index_col=None):
    donnees[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'composant_culture', 'espece', 'variete', 'intervention_synthetise', 
    'connection_synthetise', 'plantation_perenne_phases_synthetise',
    'plantation_perenne_synthetise', 'composant_culture_concerne_intervention_synthetise',
    'noeuds_synthetise', 'noeuds_synthetise_restructure', 'plantation_perenne_synthetise_restructure', 
    'domaine', 'culture', 'ccc_intervention_synthetise_restructure',
    'connection_synthetise_restructure'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

# import des données de performance
import_dfs(['intervention_synthetise_agrege', 'combinaison_outil_materiel'], ENTREPOT_PATH, sep = ',', verbose=False)


100%|██████████| 2/2 [00:10<00:00,  5.29s/it]


In [14]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_bd9a6f19-0411-4322-85c2-878f8d73afb3'
]

In [15]:
left = donnees['noeuds_synthetise']
right = donnees['noeuds_synthetise_restructure']
donnees['noeuds_synthetise'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

left = donnees['plantation_perenne_synthetise']
right = donnees['plantation_perenne_synthetise_restructure']
donnees['plantation_perenne_synthetise'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

left = donnees['connection_synthetise']
right = donnees['connection_synthetise_restructure']
donnees['connection_synthetise'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

In [16]:
donnees['intervention_synthetise_test'] = donnees['intervention_synthetise'].loc[
    donnees['intervention_synthetise'].index.isin(studied_ids)
]
donnees['connection_synthetise_test'] = donnees['connection_synthetise'].loc[
    (donnees['connection_synthetise'].index.isin(list(donnees['intervention_synthetise_test']['connection_synthetise_id'])))
]

donnees['noeuds_synthetise_test'] = donnees['noeuds_synthetise'].loc[
    (donnees['noeuds_synthetise'].index.isin(list(donnees['connection_synthetise_test']['source_noeuds_synthetise_id']))) |
    (donnees['noeuds_synthetise'].index.isin(list(donnees['connection_synthetise_test']['cible_noeuds_synthetise_id'])))
]

donnees['plantation_perenne_phases_synthetise_test'] = donnees['plantation_perenne_phases_synthetise'].loc[
    donnees['plantation_perenne_phases_synthetise'].index.isin(list(donnees['intervention_synthetise_test']['plantation_perenne_phases_synthetise_id']))
]

donnees['plantation_perenne_synthetise_test'] = donnees['plantation_perenne_synthetise'].loc[
    donnees['plantation_perenne_synthetise'].index.isin(list(donnees['plantation_perenne_phases_synthetise_test']['plantation_perenne_synthetise_id']))
]

donnees['composant_culture_test'] = donnees['composant_culture'].loc[
    donnees['composant_culture']['culture_id'].isin(list(donnees['noeuds_synthetise_test']['culture_id'])) |
    donnees['composant_culture']['culture_id'].isin(list(donnees['plantation_perenne_synthetise_test']['culture_id'])) |
    donnees['composant_culture']['culture_id'].isin(list(donnees['connection_synthetise_test']['culture_intermediaire_id']))
]

donnees['culture_test'] = donnees['culture'].loc[
    donnees['culture'].index.isin(donnees['composant_culture_test']['culture_id'])
]

donnees['espece_test'] = donnees['espece'].loc[
    donnees['espece'].index.isin(list(donnees['composant_culture_test']['espece_id']))
]
donnees['variete_test'] = donnees['variete'].loc[
    donnees['variete'].index.isin(list(donnees['composant_culture_test']['variete_id']))
]

donnees['composant_culture_concerne_intervention_synthetise_test'] = donnees['composant_culture_concerne_intervention_synthetise'].loc[
    donnees['composant_culture_concerne_intervention_synthetise']['intervention_synthetise_id'].isin(studied_ids)
]

donnees['connection_synthetise_restructure_test'] = donnees['connection_synthetise_restructure'].loc[
    donnees['connection_synthetise_restructure'].index.isin(donnees['connection_synthetise_test'].index)
]


donnees['noeuds_synthetise_restructure_test']= donnees['noeuds_synthetise_restructure'].loc[donnees['noeuds_synthetise_test'].index]
donnees['plantation_perenne_synthetise_restructure_test']= donnees['plantation_perenne_synthetise_restructure'].loc[donnees['plantation_perenne_synthetise_test'].index]
donnees['ccc_intervention_synthetise_restructure_test'] = donnees['ccc_intervention_synthetise_restructure'].loc[donnees['composant_culture_concerne_intervention_synthetise_test'].index]


donnees['plantation_perenne_synthetise_test'] = donnees['plantation_perenne_synthetise_test'].drop('culture_id', axis=1)
donnees['noeuds_synthetise_test'] = donnees['noeuds_synthetise_test'].drop('culture_id', axis=1)
donnees['connection_synthetise_test'] = donnees['connection_synthetise_test'].drop('culture_intermediaire_id', axis=1)

path=BASE_PATH+'02_outils/tests/data/test_get_intervention_synthetise_culture_outils_can2/'
donnees['intervention_synthetise_test'].to_csv(path+'intervention_synthetise'+'.csv')
donnees['noeuds_synthetise_test'].to_csv(path+'noeuds_synthetise'+'.csv')
donnees['connection_synthetise_test'].to_csv(path+'connection_synthetise'+'.csv')
donnees['composant_culture_test'].to_csv(path+'composant_culture'+'.csv')
donnees['espece_test'].to_csv(path+'espece'+'.csv')
donnees['variete_test'].to_csv(path+'variete'+'.csv')
donnees['composant_culture_concerne_intervention_synthetise_test'].to_csv(path+'composant_culture_concerne_intervention_synthetise'+'.csv')
donnees['culture_test'].to_csv(path+'culture'+'.csv')
donnees['plantation_perenne_phases_synthetise_test'].to_csv(path+'plantation_perenne_phases_synthetise'+'.csv')
donnees['plantation_perenne_synthetise_test'].to_csv(path+'plantation_perenne_synthetise'+'.csv')
donnees['noeuds_synthetise_restructure_test'].to_csv(path+'noeuds_synthetise_restructure'+'.csv')
donnees['plantation_perenne_synthetise_restructure_test'].to_csv(path+'plantation_perenne_synthetise_restructure'+'.csv')
donnees['ccc_intervention_synthetise_restructure_test'].to_csv(path+'ccc_intervention_synthetise_restructure'+'.csv')
donnees['connection_synthetise_restructure_test'].to_csv(path+'connection_synthetise_restructure'+'.csv')